In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# base_dir = os.path.dirname(os.path.dirname(__file__))
# data_dir = os.path.join(base_dir, "data")
fixed_csv = pd.read_csv("SAARTHI V1.2.0\task_manager\data\fixed_schedule_5min.csv")
sleep_csv = os.path.join(data_dir, "sleep.csv")
output_csv = os.path.join(data_dir, "grouped_schedule.csv")

<>:3: SyntaxWarning: invalid escape sequence '\ '
<>:3: SyntaxWarning: invalid escape sequence '\ '
C:\Users\DELL\AppData\Local\Temp\ipykernel_38672\3944297480.py:3: SyntaxWarning: invalid escape sequence '\ '
  fixed_csv = pd.read_csv("SAARTHI V1.2.0\task_manager\ data\fixed_schedule_5min.csv")
C:\Users\DELL\AppData\Local\Temp\ipykernel_38672\3944297480.py:3: SyntaxWarning: invalid escape sequence '\ '
  fixed_csv = pd.read_csv("SAARTHI V1.2.0\task_manager\ data\fixed_schedule_5min.csv")


OSError: [Errno 22] Invalid argument: 'SAARTHI V1.2.0\task_manager\\ data\x0cixed_schedule_5min.csv'

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

def expand_to_5_min_slots(day, start_str, end_str, task):
    fmt = "%H:%M"

    # Skip if any value is missing
    if pd.isna(start_str) or pd.isna(end_str):
        return []

    # Ensure the values are strings
    start_str = str(start_str)
    end_str = str(end_str)

    start_time = datetime.strptime(start_str, fmt)
    end_time = datetime.strptime(end_str, fmt)

    slots = []
    current = start_time
    while current < end_time:
        slot_end = current + timedelta(minutes=5)
        slots.append({
            "Day": day,
            "Start": current.strftime("%H:%M"),
            "End": slot_end.strftime("%H:%M"),
            "Task": task
        })
        current = slot_end
    return slots

In [ ]:
def group_slots_by_task(df):
    df['Start_dt'] = pd.to_datetime(df['Start'], format="%H:%M")
    df['End_dt'] = pd.to_datetime(df['End'], format="%H:%M")

    df = df.sort_values(by=["Day", "Start_dt"]).reset_index(drop=True)

    group_ids = [0]
    for i in range(1, len(df)):
        same_task = df.loc[i, 'Task'] == df.loc[i - 1, 'Task']
        same_day = df.loc[i, 'Day'] == df.loc[i - 1, 'Day']
        continuous = df.loc[i, 'Start_dt'] == df.loc[i - 1, 'End_dt']
        if same_task and same_day and continuous:
            group_ids.append(group_ids[-1])
        else:
            group_ids.append(group_ids[-1] + 1)

    df['Group'] = group_ids
    grouped = df.groupby(['Day', 'Group', 'Task']).agg({'Start_dt': 'min', 'End_dt': 'max'}).reset_index()
    grouped['Start'] = grouped['Start_dt'].dt.strftime("%H:%M")
    grouped['End'] = grouped['End_dt'].dt.strftime("%H:%M")

    grouped['Day'] = pd.Categorical(grouped['Day'], categories=[
        "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
    ], ordered=True)

    return grouped[['Day', 'Start', 'End', 'Task']]


In [ ]:
# task_manager/schedular/load_day_view.py
def load_and_expand(schedule_df, day):
    filtered = schedule_df[schedule_df["Day"] == day]
    if filtered.empty:
        print(f"❌ No tasks found for {day}")
    else:
        print(f"\n📅 Schedule for {day}")
        print(filtered.to_string(index=False))


In [ ]:
def build_schedule():
    # Load fixed tasks and sleep
    fixed_df = pd.read_csv(fixed_csv)
    sleep_df = pd.read_csv(sleep_csv)

    # Add 'Task' column to sleep_df if missing
    if "Task" not in sleep_df.columns:
        sleep_df["Task"] = "Sleep"

    # Combine both
    df = pd.concat([fixed_df, sleep_df], ignore_index=True)

    # Expand to 5-min slots
    expanded_records = []
    for _, row in df.iterrows():
        if pd.isna(row["Start"]) or pd.isna(row["End"]):
            continue
        expanded_records.extend(
            expand_to_5_min_slots(str(row["Day"]), str(row["Start"]), str(row["End"]), str(row["Task"]))
        )
    expanded_df = pd.DataFrame(expanded_records)

    # Group and save
    grouped_df = group_slots_by_task(expanded_df)
    grouped_df.to_csv(output_csv, index=False)

    return grouped_df

def view_day_schedule(day_name):
    grouped_df = build_schedule()
    load_and_expand(grouped_df, day_name)

In [ ]:
def show_day_schedule():
    day = input("📅 Enter the day you want to view (e.g., Monday): ").capitalize()
    schedule_df = view_day_schedule(day)
    load_and_expand(schedule_df, day)

In [ ]:
show_day_schedule()